# NER Task
개체명 인식(Named Entity Recognition, NER)은 자연어 텍스트에서 사람, 기관, 지명 등의 단어열을 뽑아내는 task입니다.

<img src="files/ner1.PNG">

텍스트를 형태소 단위로 분해한 뒤, 개체의 범위를 Begin, Inside, Outside를 나타내는 BIO 태깅을 통해 표현합니다. 

BIO 태그와 함께 개체의 타입을 맞추는 것이 개체명 인식의 최종 목적입니다. 태그에 대한 예시는 [예시 파일](corpus/parsed.txt)을 참고해 주세요.
<img src="files/ner2.PNG">


이 task에서는 Word index 기반과 Word embedding 기반으로 간단한 Bi-directional LSTM을 사용하여 개체명 인식 모듈을 만들어 보겠습니다.

모듈은 pytorch 기반으로, pytorch 모듈은 [이 노트북](cifar10_tutorial.ipynb)를 참고해 주세요

In [1]:
# import modules
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from konlpy.tag import Okt
from seqeval.metrics import f1_score
from tqdm import tqdm

from data import parse, one_hot_batch, decode

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## NER based on Word Index

Word index를 받은 뒤, 그 index를 one-hot vector로 만듭니다.

One-hot vector는 `self.lstm`을 통해 token 주변의 context를 가지게 됩니다.[(참고)](https://pytorch.org/docs/stable/nn.html#lstm)
lstm의 입력으로 길이가 다른 문장을 받게 되는데, 이를 고려하기 위해 `pack_padded_sequence`라는 함수를 사용합니다.
<img src="https://dl.dropbox.com/s/3ze3svhdz05aakk/0705img3.gif">

`self.lstm`의 출력값을 `self.ffnn`을 통과시켜 최종 tag를 얻습니다.

In [3]:
class WordIndexBasedNER(nn.Module):
    def __init__(self, word_input_size, hidden_size, tag_size):
        super(WordIndexBasedNER, self).__init__()
        self.lstm = nn.LSTM(input_size=word_input_size, hidden_size=hidden_size, num_layers=1, batch_first=True, bidirectional=True)
        self.ffnn = nn.Linear(hidden_size * 2, tag_size)
        self.tk_len = word_input_size
    
    def forward(self, input_batch, word_lens, labels=None):
        input_batch = torch.stack([torch.tensor(one_hot_batch(x, self.tk_len)) for x in input_batch]).to(device, dtype=torch.float32)
        input_batch = nn.utils.rnn.pack_padded_sequence(input_batch, word_lens, batch_first=True, enforce_sorted=False)
        out, _ = self.lstm(input_batch)
        out, output_lens = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        out = F.dropout(out)
        pred = self.ffnn(out)
        
        if labels is not None:
            # 정답이 주어진 경우: loss를 return
            pb = torch.zeros(torch.sum(output_lens), pred.size()[-1])
            lb = torch.zeros(torch.sum(output_lens), dtype=torch.long)
            lsum = 0
            for p, la, le in zip(pred, labels, output_lens):
                pb[lsum:lsum+le] = p[:le, :]
                lb[lsum:lsum+le] = la[:le]
                lsum += le
            pred = pb
            labels = lb
            
            loss = F.cross_entropy(pred, labels)
            return loss
        else:
            # 정답이 주어지지 않은 경우: prediction을 return
            pred = F.softmax(pred, dim=-1)
            pred = torch.argmax(pred, dim=-1)
            return pred

## NER based on Word Embedding

Word index를 받은 뒤, `self.we`에서 정의된 word embedding의 row를 불러 옵니다.

해당 row에는 그 단어의 word embedding이 정의되어 있습니다.

Word embedding을 `self.lstm`을 통과시킵니다. 이렇게 하면 각각의 단어에 대해 contextualized embedding을 얻을 수 있습니다.

마지막으로, `self.lstm`에서 얻은 token의 embedding을 feedforward network인 `self.ffnn`을 통과시켜 각각의 단어에 대한 tag를 얻을 수 있습니다.

In [4]:
class WordEmbeddingBasedNER(nn.Module):
    def __init__(self, we, hidden_size, tag_size):
        super(WordEmbeddingBasedNER, self).__init__()
        self.we = nn.Embedding.from_pretrained(torch.FloatTensor(we))
        self.lstm = nn.LSTM(input_size=self.we.embedding_dim, hidden_size=hidden_size, num_layers=1, batch_first=True, bidirectional=True)
        self.ffnn = nn.Linear(hidden_size * 2, tag_size)
        
    
    def forward(self, input_batch, word_lens, labels=None):
#         wi, idx = word_lens.sort(0, descending=True)
        input_batch = self.we(input_batch)
#         input_batch = torch.index_select(input_batch, 0, idx)
        input_batch = nn.utils.rnn.pack_padded_sequence(input_batch, word_lens, batch_first=True, enforce_sorted=False)
        out, _ = self.lstm(input_batch)
        out, output_lens = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        out = F.dropout(out)
        pred = self.ffnn(out)
        
        if labels is not None:
            pb = torch.zeros(torch.sum(output_lens), pred.size()[-1])
            lb = torch.zeros(torch.sum(output_lens), dtype=torch.long)
            lsum = 0
            for p, la, le in zip(pred, labels, output_lens):
                pb[lsum:lsum+le] = p[:le, :]
                lb[lsum:lsum+le] = la[:le]
                lsum += le
            pred = pb
            labels = lb
#             pred = pred.view(-1, pred.size()[-1])
            
            loss = F.cross_entropy(pred, labels)
            return loss
        else:
            pred = F.softmax(pred, dim=-1)
            pred = torch.argmax(pred, dim=-1)
            return pred
#         return pred, output_lens

### Dataset generation
[torch data module](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)을 사용하여 data를 loading하는 구조체를 만듭니다.

Dataset은 data를 저장하고 index-based로 data의 tensor화된 자료를 얻어오게 됩니다. 따라서 데이터를 tensor로 변환하는 과정이 필요합니다.

Data는 word와 tag로 이루어져 있는데, 이 task에서의 dataset은 각각의 data에 대해 *word index*, *tag index*, *token length*를 return합니다.

In [5]:
class NERDataset(torch.utils.data.Dataset):
    def __init__(self, data, token_index_dict, tag_index_dict):
        self.data = data
        self.token2i = token_index_dict
        self.tag2i = tag_index_dict
        self.maxlen = max(map(len, data))
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        data = self.data[index]
        return torch.tensor([self.token2i[x] if x in self.token2i else 0 for x, _ in data] + ([0] * (self.maxlen - len(data)))), \
                torch.tensor([self.tag2i[x] for _, x in data] + ([0] * (self.maxlen - len(data)))), \
                len(data)

### Load corpus and embedding
학습, 테스트용 corpus와 모델에 사용할 Word embedding 파일을 불러옵니다.

Word embedding의 경우, 학습되지 않은 단어에 대한 임베딩은 0벡터를 부여합니다.

In [6]:
parsed_data = parse("corpus/parsed.txt")
train_data = parsed_data[:-1000]
dev_data = parsed_data[-1000:]

In [7]:
tok_ids = {"_UNK_": 0} # 0번 id에 더미 입력
with open("wiki_tok_glove_300.word", encoding="UTF8") as f:
    for line in f.readlines():
        tok_ids[line.strip()] = len(tok_ids)
we = np.load("wiki_tok_glove_300.npy")
we = np.vstack([np.zeros([1, we.shape[1]]), we] # 0번 row에 0벡터 입력
               
print(tok_ids["하이닉스"], we[tok_ids["하이닉스"]])

In [8]:
# make token/tag dictionary
token_dict = set([])
tag_dict = set([])
for sent in train_data+dev_data:
    for token, tag in sent:
        token_dict.add(token)
        tag_dict.add(tag)

token_dict = {t: i for i, t in enumerate(token_dict)}
tag_dict = {t: i for i, t in enumerate(tag_dict)}

i2tag = {i: t for t, i in tag_dict.items()}

In [9]:
print(len(tok_ids), len(token_dict), len(tag_dict))
print(tag_dict)
print(i2tag)

504227 26256 11
{'I-DT': 0, 'B-PS': 1, 'I-OG': 2, 'B-DT': 3, 'I-PS': 4, 'I-TI': 5, 'B-TI': 6, 'B-LC': 7, 'I-LC': 8, 'O': 9, 'B-OG': 10}
{0: 'I-DT', 1: 'B-PS', 2: 'I-OG', 3: 'B-DT', 4: 'I-PS', 5: 'I-TI', 6: 'B-TI', 7: 'B-LC', 8: 'I-LC', 9: 'O', 10: 'B-OG'}


In [10]:
print(train_data[0])

[('\ufeff', 'O'), ('특히', 'O'), ('김병현', 'B-PS'), ('은', 'O'), ('4회', 'O'), ('말에', 'O'), ('무기', 'O'), ('력', 'O'), ('하게', 'O'), ('6', 'O'), ('실점', 'O'), ('하면서', 'O')]


Module과 optimizer를 정의합니다. Word index 기반 모듈을 `ner_wi_module`, embedding 기반 모듈을 `ner_we_module`로 정의합니다. `self.lstm`의 hidden layer size는 256으로 설정했습니다.

Optimizer의 경우, Adam optimizer를 사용합니다. optimizer에는 각각의 모듈의 parameter가 학습 대상임을 알려주고, learning rate를 정의합니다.

In [11]:
# initialize module
ner_wi_module = WordIndexBasedNER(len(token_dict), 256, len(tag_dict)).to(device)
ner_we_module = WordEmbeddingBasedNER(we, 256, len(tag_dict)).to(device)
# ner_parallel = torch.DataParallel(ner_module)
wi_optimizer = torch.optim.Adam(ner_wi_module.parameters(), lr=0.0001)
we_optimizer = torch.optim.Adam(ner_we_module.parameters(), lr=0.0001)

### Train Word index based module
Train 과정은 다음과 같이 이루어집니다.
1. Batch load
2. Prediction
3. 정답과 비교 후 loss 계산
4. Backpropagation
5. Evaluation score 계산

Data loading은 [Cell 5](#Dataset-generation) 에서 정의한 dataset 모듈을 바탕으로 한 DataLoader 모듈을 사용하여 이루어집니다. DataLoader는 Dataset에서 정의된 output tensor들을 여러 변환을 통해 가져올 수 있는 모듈입니다.

module은 train mode와 eval mode가 있습니다. train시에는 dropout과 같은 train과정에만 적용되어야 하는 것들이 적용되고, eval시에는 해당 부분들이 동작하지 않습니다.

loss backpropagation은 prediction을 구한 뒤, label과 비교하여 loss를 얻고, `loss.backward()`, `optimizer.step()`을 통해 backpropagation을 수행합니다.(해당 코드는 모듈 내 구현되어 있습니다. 모듈의 `forward()`함수에서 `if labels is not None:` 부분이 label과 비교하여 loss를 구하는 부분입니다.)

In [12]:
# train iteration
max_epoch = 0
eval_per_epoch = 2

train_dataset = NERDataset(train_data, token_dict, tag_dict)
dev_dataset = NERDataset(dev_data, token_dict, tag_dict)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
dev_dataloader = DataLoader(dev_dataset, shuffle=False, batch_size=32)

best_f1 = 0
best_epoch = 0
nbc = 0
tq = tqdm(range(1, max_epoch+1))
for epoch in tq:
    ner_wi_module.train()
    total_loss = 0
    for token_batch, tag_batch, data_len in train_dataloader: #1
        wi_optimizer.zero_grad()
        token_batch, tag_batch, data_len = token_batch.to(device), tag_batch.to(device), data_len.to(device)
        max_data_len = torch.max(data_len)
        loss = ner_wi_module(token_batch, data_len, tag_batch) #2, #3
        total_loss += loss
        loss.backward() #4
        wi_optimizer.step() #4
    tq.desc = "Epoch %d Loss %f" % (epoch, total_loss)
    if epoch % eval_per_epoch == 0: #Evaluation mode
        ner_wi_module.eval()
        preds = []
        golds = []
        lens_with_pad = []
        data_lens = []
        for token_batch, tag_batch, data_len in dev_dataloader:
            
            token_batch, data_len = token_batch.to(device), data_len.to(device)
            pred = ner_wi_module(token_batch, data_len)
            for p, t, d in zip(pred, tag_batch, data_len):
                preds.append([i2tag[x.item()] for x in p[:d]])
                golds.append([i2tag[x.item()] for x in t[:d]])
                data_lens.append(d)
        f1 = f1_score(golds, preds) #5
        nbc += eval_per_epoch
        if f1 > best_f1:
            best_f1 = f1
            best_epoch = epoch
            torch.save(ner_wi_module.state_dict(), "wi_model")
            nbc = 0
        
        idx = 0
        with open("debug/wi_eval_%d.tsv" % epoch, "w", encoding="UTF8") as f:
            for b in dev_data:
                
                for i, (token, tag) in enumerate(b):
                    assert golds[idx][i] == tag
                    f.write("\t".join([token, tag, preds[idx][i]])+"\n")
                f.write("\n")
                idx += 1
        print("Epoch %d F1 score %.2f" % (epoch, f1 * 100))
        print("Best F1 %.2f at Epoch %d" % (best_f1 * 100, best_epoch))
        if nbc >= 30:
            print("No better result since epoch %d - stop training" % best_epoch)
            break

0it [00:00, ?it/s]


### Train word embedding based module

In [13]:
from evaluation import eval_ner
# train iteration
max_epoch = 1000
eval_per_epoch = 5

train_dataset = NERDataset(train_data, tok_ids, tag_dict)
dev_dataset = NERDataset(dev_data, tok_ids, tag_dict)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
dev_dataloader = DataLoader(dev_dataset, shuffle=False, batch_size=32)

best_f1 = 0
best_epoch = 0
nbc = 0
# ner_parallel = nn.DataParallel(ner_module)
tq = tqdm(range(1, max_epoch+1))
for epoch in tq:
    ner_we_module.train()
    total_loss = 0
    for token_batch, tag_batch, data_len in train_dataloader:
        we_optimizer.zero_grad()
        token_batch, tag_batch, data_len = token_batch.to(device), tag_batch.to(device), data_len.to(device)
        max_data_len = torch.max(data_len)
        loss = ner_we_module(token_batch, data_len, tag_batch)
        total_loss += loss
        loss.backward()
        we_optimizer.step()
    tq.desc = "Epoch %d Loss %f" % (epoch, total_loss)
    if epoch % eval_per_epoch == 0:
        ner_we_module.eval()
        preds = []
        golds = []
        lens_with_pad = []
        data_lens = []
        for token_batch, tag_batch, data_len in dev_dataloader:
            
            token_batch, data_len = token_batch.to(device), data_len.to(device)
            pred = ner_we_module(token_batch, data_len)
            for p, t, d in zip(pred, tag_batch, data_len):
                preds.append([i2tag[x.item()] for x in p[:d]])
                golds.append([i2tag[x.item()] for x in t[:d]])
                data_lens.append(d)
        f1 = f1_score(golds, preds)
        nbc += eval_per_epoch
        if f1 > best_f1:
            best_f1 = f1
            best_epoch = epoch
            torch.save(ner_we_module.state_dict(), "we_model")
            nbc = 0
        
        idx = 0
        with open("debug/we_eval_%d.tsv" % epoch, "w", encoding="UTF8") as f:
            for b in dev_data:
                
                for i, (token, tag) in enumerate(b):
                    assert golds[idx][i] == tag
                    f.write("\t".join([token, tag, preds[idx][i]])+"\n")
                f.write("\n")
                idx += 1
        print("Epoch %d F1 score %.2f" % (epoch, f1 * 100))
        print("Best F1 %.2f at Epoch %d" % (best_f1 * 100, best_epoch))
        if nbc >= 30:
            print("No better result since epoch %d - stop training" % best_epoch)
            break

Epoch 5 Loss 99.065842:   0%|          | 5/1000 [00:50<2:57:35, 10.71s/it] 

Epoch 5 F1 score 34.50
Best F1 34.50 at Epoch 5


Epoch 10 Loss 77.676620:   1%|          | 10/1000 [01:41<3:00:20, 10.93s/it]

Epoch 10 F1 score 40.92
Best F1 40.92 at Epoch 10


Epoch 15 Loss 65.804337:   2%|▏         | 15/1000 [02:32<3:01:17, 11.04s/it]

...

Epoch 50 F1 score 45.22
Best F1 45.32 at Epoch 30


Epoch 55 Loss 15.337384:   6%|▌         | 55/1000 [08:59<2:31:14,  9.60s/it]

Epoch 55 F1 score 44.91
Best F1 45.32 at Epoch 30


Epoch 59 Loss 13.038646:   6%|▌         | 59/1000 [09:36<2:24:43,  9.23s/it]

Epoch 60 F1 score 44.17
Best F1 45.32 at Epoch 30
No better result since epoch 30 - stop training


### Application
실제 학습된 모델을 적용해봅니다. 원문 sentence가 들어왔을 때, sentence를 형태소 분석을 하여 단어의 의미를 더 잘 표현하게 해 주고, 이를 model에 넣어서 tag 예측을 한 뒤 BIO 태그를 decode합니다.

In [14]:
okt = Okt()
def sentence_ner(sentence):
    morphs = okt.morphs(sentence)
    word_indexes = torch.LongTensor([tok_ids[morph] if morph in tok_ids else 0 for morph in morphs]).to(device).unsqueeze(0)
    word_len = torch.LongTensor([len(morphs)])
    pred = ner_we_module(word_indexes, word_len)
    pred = [i2tag[x.item()] for x in pred[0]]
    print(decode(morphs, pred))

sentence_ner("7월 10일 SK 와이번스는 한화 이글스를 상대로 승리하였다.")

<7월:PS> 10일 <SK 와이번스:OG> 는 <한화 이글스:OG> 를 상대로 승리 하였다 .
